In [1]:
import sys
import os
import pandas as pd
sys.path.append("../src")
import json
from data import data
from constants import Timeframe
from datetime import datetime
from pydantic import BaseModel, ValidationError, PositiveInt
from datetime import datetime
from backtesting.backtest_config import BacktestConfig
from pydantic import BaseModel

pd.set_option('display.max_columns', None)

In [3]:
df = data.get_df("ADAUSD", Timeframe.H1, ".", 'PARQUET')
df.head(10)

,timestamp,open,high,low,close,vwap,volume,count
0,06-14-2025 08:00,0.636120,0.636157,0.632631,0.635779,0.634438,72958.05845170,100
1,06-14-2025 09:00,0.635779,0.637307,0.633689,0.634201,0.636055,8455.59340953,60
2,06-14-2025 10:00,0.634201,0.635107,0.631254,0.631590,0.632340,209220.49432924,118
3,06-14-2025 11:00,0.631644,0.632468,0.626787,0.628742,0.628676,46561.52882625,149
4,06-14-2025 12:00,0.628456,0.630054,0.615000,0.618572,0.619028,1291097.81141900,413
5,06-14-2025 13:00,0.618213,0.624343,0.617405,0.622423,0.621914,916174.83432613,259
6,06-14-2025 14:00,0.622422,0.625306,0.616001,0.620680,0.619486,37015.52017732,166
7,06-14-2025 15:00,0.620634,0.622981,0.619312,0.619357,0.620857,51525.44255249,105
8,06-14-2025 16:00,0.619313,0.619783,0.613435,0.618489,0.617098,141801.23402422,264
9,06-14-2025 17:00,0.618646,0.626417,0.618271,0.626364,0.622262,39796.89686537,137


In [ ]:
class Trade(BaseModel):
    # price of quote crypto
    open_price: float = 0.0
    # total value of quote crypto in base
    open_value: float = 0.0
    # trade size of base crypto/currency(USD)
    total_quantity: float = 0.0
    # price of quote crypto
    current_price: float = 0.0
    # profit/loss in base crypto/currency(USD)
    profit_loss: float = 0.0
    # profit/loss percentage
    profit_loss_pct: float = 0.0
    # value of quote crypto in base
    current_value: float = 0.0
    averaging_orders_prices: list = []
    averaging_order_sizes: list = []
    # list of the averaging order quantities (order_size / price), how many of the quote did you get
    averaging_orders_qty: list = []
    averaging_orders_filled: int = 0
    last_order_price: float = 0.0

def calc_avgeraging_order_prices(initial_price:float, deviation:float, multiplier:float):
        prices = []
        cumulative_deviation = 0

        for i in range(6):
            cumulative_deviation += deviation
            price = initial_price * (1 - (cumulative_deviation / 100))
            prices.append(round(price, 7))
        return prices

def calc_averaging_order_size(avg_order_size: float, multiplier: float):
     sizes = [
          round(avg_order_size * (multiplier ** i), 2)
          for i in range(6)
     ]
     return sizes

def calc_profit_losses(current_price: float, total_cost: float, total_qty: float):
    current_value = current_price * total_qty
    profit_loss = current_value - total_cost
    profit_loss_pct = (profit_loss / total_cost) * 100

    return round(profit_loss, 6), round(profit_loss_pct, 6)
     

def run_backtest(df):
    df_copy = df.copy().drop(columns=['vwap','volume','count'])
    df_copy['trade_action'] = 'no_trade'
    df_copy['trade_value'] = 'None'
    df_copy['trade_profit_loss'] = 0.0
    df_copy['trade_profit_loss_pct'] = 0.0

    trades: list[Trade] = []

    current_trade = Trade()

    total_profit = 0.0
    hit_take_profit = False
    
    #bot config
    base_order_size = 20
    averaging_order_size = 40
    max_averaging_orders = 6
    take_profit = 1.5
    price_deviation = 1.5
    averaging_order_size_mul = 1.28
    averaging_order_step_mul = 1.85

    #account info
    starting_account_balance = 500.0
    current_account_balance = 500.0

    for index, row in enumerate(df_copy.itertuples(), 0):
        if current_trade.profit_loss_pct > take_profit:
            total_profit += current_trade.profit_loss
            df_copy.loc[index, 'trade_action'] = 'closed_trade'
            df_copy.loc[index + 1, 'trade_action'] = 'no_trade'
            current_trade = Trade()
        elif row.trade_action == 'in_trade':
            # maintain trade
            avg_orders_filled = current_trade.averaging_orders_filled
            if float(row.low) < current_trade.averaging_orders_prices[avg_orders_filled] and avg_orders_filled < 5:
                 current_trade.last_order_price = current_trade.averaging_orders_prices[avg_orders_filled]
                 current_trade.total_quantity += (current_trade.averaging_order_sizes[avg_orders_filled] / float(row.low))
                 current_trade.open_value += current_trade.averaging_order_sizes[avg_orders_filled]

                 df_copy.loc[index, 'trade_action'] = f"avg. order {avg_orders_filled + 1} filled"
                 if (avg_orders_filled < 5): current_trade.averaging_orders_filled += 1
            current_trade.current_value = float(current_trade.total_quantity * float(row.open))
            
            # current_price_deviation = ((float(row.open) - current_trade.last_order_price) / current_trade.last_order_price) * 100

            profit_loss, profit_loss_pct = calc_profit_losses(
                 current_price=float(row.open), 
                 total_cost=current_trade.open_value, 
                 total_qty=current_trade.total_quantity
                 )
            current_trade.profit_loss = profit_loss
            current_trade.profit_loss_pct = profit_loss_pct

            df_copy.loc[index, 'trade_value'] = current_trade.current_value
            df_copy.loc[index, 'trade_profit_loss'] = current_trade.profit_loss
            df_copy.loc[index, 'trade_profit_loss_pct'] = current_trade.profit_loss_pct
            df_copy.loc[index + 1, 'trade_action'] = 'in_trade'

        elif row.trade_action == 'no_trade':
            # open trade
            current_trade.total_quantity = base_order_size / float(row.open)
            current_trade.open_value = float(current_trade.total_quantity * float(row.open))
            current_trade.last_order_price = float(row.open)
            current_trade.averaging_orders_prices = calc_avgeraging_order_prices(
                initial_price=float(row.open),
                deviation=price_deviation,
                multiplier=averaging_order_step_mul
                )
            current_trade.averaging_order_sizes = calc_averaging_order_size(
                 avg_order_size=averaging_order_size,
                 multiplier=averaging_order_size_mul
            )
            
            df_copy.loc[index, 'trade_action'] = 'opened_trade'
            df_copy.loc[index, 'trade_value'] = current_trade.open_value
            df_copy.loc[index + 1, 'trade_action'] = 'in_trade'
    trades.append(current_trade)

    return df_copy, trades, total_profit

In [ ]:
df_tested, trades, profit = run_backtest(df)
print(profit)
df_tested.head(60)

In [ ]:
%%timeit
run_backtest(df)

# Testing Ways to Loop or Vectorize DF:

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def get_data(size = 10_000):
    df = pd.DataFrame()
    df['age'] = np.random.randint(0, 100, size)
    df['time_in_bed'] = np.random.randint(0, 9, size)
    df['pct_sleeping'] = np.random.rand(size)
    df['favorite_food'] = np.random.choice(['pizza', 'taco', 'ice-cream'], size)
    df['hate_food'] = np.random.choice(['broccoli', 'candy corn', 'eggs'], size)
    return df

## The problem
Reqard calc:
- If they were in bed for more than 5 hours AND they were sleeping for more than 50% we give them their favorite food.
- Otherwise we give them their hate food.
- If they are over 90 years old give their favorite foor regardless

In [ ]:
def reward_calc(row):
    if row['age'] >= 90:
        return row['favorite_food']
    if (row['time_in_bed'] > 5) & (row['pct_sleeping'] > 0.5):
        return row['favorite_food']
    return row['hate_food']

### Level 1 - Loop (Slowest)

In [ ]:
df = get_data()

In [ ]:
%%timeit
for index, row in df.iterrows():
    df.loc[index, 'reward'] = reward_calc(row)

### Level 2 - Apply

In [ ]:
df = get_data()

In [ ]:
%%timeit
df['reward'] = df.apply(reward_calc, axis=1)

### Level 3 - Vectorized (Fastest)

In [ ]:
df = get_data()

In [ ]:
%%timeit
df['reward'] = df['hate_food']
df.loc[
    ((df['pct_sleeping'] > 0.5) & (df['time_in_bed'] > 5) | (df['age'] > 90)), 
    'reward'
    ] = df['favorite_food']